In [129]:
using LinearAlgebra
using SparseArrays
using Printf

In [130]:
import MathProgBase
MPB = MathProgBase

import Mosek
import Mosek:MosekSolver

MPB.getbarrieriter(m::Mosek.MosekMathProgSolverInterface.MosekLinearQuadraticModel) = Mosek.getintinf(m.task, Mosek.MSK_IINF_INTPNT_ITER)
MPB.getsimplexiter(m::Mosek.MosekMathProgSolverInterface.MosekLinearQuadraticModel) = Mosek.getintinf(m.task,Mosek.MSK_IINF_SIM_PRIMAL_ITER)+Mosek.getintinf(m.task,Mosek.MSK_IINF_SIM_DUAL_ITER)

In [355]:
import Gurobi
MPB.getsimplexiter(m::Gurobi.GurobiMathProgModel) = Gurobi.get_dblattr(m.inner, "IterCount")
MPB.getbarrieriter(m::Gurobi.GurobiMathProgModel) = Gurobi.get_intattr(m.inner, "BarIterCount")

In [131]:
solver = MosekSolver(MSK_IPAR_LOG=0);

In [132]:
import Linda

# TSSP formulation

Deterministic equivalent:

\begin{align*}
    (DEP) \ \ \ \min_{x, y} \ \ \ & c^{T}x + \sum_{i} p_{i} q_{i}^{T} y_{i}\\
    s.t. \ \ \ 
    & Ax = b,\\
    & T_{i} x + W_{i}y_{i} = h_{i}, \ \ \ \forall i,\\
    & x \geq 0,\\
    & y_{i} \geq 0, \ \ \ \forall i.
\end{align*}

Dual of (DEP) writes:
\begin{align*}
    (D) \ \ \ \max_{\eta, \theta} \ \ \ & b^{T}\eta + \sum_{i} h_{i}^{T} \theta_{i}\\
    s.t. \ \ \ 
    & A^{T} \eta + \sum_{i} T_{i}^{T} \theta_{i} \leq c,\\
    & W_{i}^{T} \theta_{i} \leq p_{i} q_{i}, \ \ \ \forall i.
\end{align*}

We wolve (D) by column generation. Note that it amounts to the same as solving (DEP) by Benders.

The Master problem writes
\begin{align*}
    (D) \ \ \ \min_{\eta, \theta} \ \ \ & -b^{T}\eta - \sum_{i, \omega} h_{i, \omega} \lambda_{i, \omega} - \sum_{i, \rho} h_{i, \rho} \lambda_{i, \rho}\\
    s.t. \ \ \ 
    & A^{T} \eta + \sum_{i, \omega} t_{i, \omega} \lambda_{i, \omega} + \sum_{i, \rho} t_{i, \rho} \lambda_{i, \rho} \leq c, & [\pi]\\
    & \sum_{i} \lambda_{i, \omega} = 1, \ \ \ \forall i, & [\sigma_{i}]\\
    & \lambda \geq 0.
\end{align*}

and the sub-problem writes:
\begin{align*}
    (SP) \ \ \ \min_{\theta_{i}} \ \ \ & (-h_{i}^{T} - \pi^{T} T_{i}^{T}) \theta_{i} - \sigma_{i}\\
    s.t. \ \ \ 
    & W_{i}^{T} \theta_{i} \leq p_{i} q_{i}.
\end{align*}

# Source code

In [133]:
include("src/TSSP/parser/core_parser.jl")
include("src/TSSP/parser/time_parser.jl")
include("src/TSSP/parser/stoch_parser.jl")

parse_block_line! (generic function with 1 method)

In [134]:
function generate_scenario_data(
        A_ref, T_ref, W_ref, q_ref, h_ref,
        row2idx, col2idx, 
        prob, coeffs
)
    m1, n1 = size(A_ref)
    m2, n2 = size(W_ref)
    m2 == size(T_ref, 1) || error("T and W has different number of rows.")
    n1 == size(T_ref, 2) || error("T and A has different number of columns.")
    
    # Create problem data
    h_ = copy(h_ref)          # Right-hand side
    q_ = copy(q_ref)  # Objective ()
    
    T_ = copy(T_ref)          # T_r matrix
    W_ = copy(W_ref)          # W_r
    
    
    for (k, v) in coeffs
        row::Int = row2idx[k[1]]
        col::Int = col2idx[k[2]]
        
        row == 0 || row > m1 || error("Entry given for first-stage row $(k[1])")
        
        # Check four cases
        if row == 0 && col > 0
            # Objective coefficient
#             println("Obj: $k: $v")
            q_[col - n1] = v
            
        elseif row > 0 && col == 0
            # Right-hand side
#             println("Rhs: $k \t $v")
            h_[row - m1] = v
            
        elseif row > 0 && col > 0
            # Check whether belongs to T or W
            if col <= n1
                T_[row - m1, col] = v
            else
                W_[row - m1, col - n1] = v
            end
        else
            eror("Wrong combination of row/col indices: $k")
        end
    end
    
    q_ .*= prob
    
    return T_, W_, q_, h_
end

generate_scenario_data (generic function with 1 method)

# Test instance files

In [466]:
f_cor = "data/tssp/4node.cor"
f_sto = "data/tssp/4node.sto.4096"
f_tim = "data/tssp/4node.tim"

"data/tssp/4node.tim"

# Parse test instance

## Parse `.cor` file

In [467]:
# Extract data from .cor file
d, row2idx, col2idx, I, J, V, objI, objV, rhsI, rhsV, senses = parse_cor_file(f_cor);

## Parse `.tim` file

In [468]:
name, is_lp, col_periods, row_periods = parse_time_file(f_tim);

In [469]:
# Extract indices of first/second period rows/columns
col1, col2 = (
    col2idx[col_periods[1]]:(col2idx[col_periods[2]]-1),  # First-period variables
    col2idx[col_periods[2]]:d["ncols"]  # Second-period variables
)
row1, row2 = (
    row2idx[row_periods[1]]:(row2idx[row_periods[2]]-1),  # First-period rows
    row2idx[row_periods[2]]:d["nrows"]  # Second-period variables
)

if row1[1] == 0
    row1 = 1:row1[end]
end
if col1[1] == 0
    col1 = 1:col1[end]
end

In [470]:
# Extract vector of objective coefficients
obj = sparsevec(objI, objV, d["ncols"])
c, q = Vector(obj[col1]), Vector(obj[col2])

([50.0, 65.0, 45.0, 70.0, 60.0, 55.0, 50.0, 60.0, 55.0, 40.0  …  76.8, 57.6, 62.4, 67.2, 57.6, 38.4, 43.2, 57.6, 43.2, 62.4], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])

In [471]:
# Extract right-hand side vectors
rhs = sparsevec(rhsI, rhsV, d["nrows"])
b, h = Vector(rhs[row1]), Vector(rhs[row2])

([25.0, 25.0, 25.0, 25.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 480.0, 240.0, 0.0, 0.0], [5.0, 8.0, 4.0, 4.5, 6.7, 4.2, 10.1, 8.3, 12.2, 3.2  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])

In [472]:
# Extract constraint matrices
A_ = sparse(I, J, V);

A = A_[row1, col1]
T = A_[row2, col1]
W = A_[row2, col2];

### Convert `A` to standard form

In [473]:
n_ctr = sum(senses[row1] .!= "E")  # Number of slack variables to be added
println("n_ctr = $n_ctr")
m1 = length(row1)
m2 = length(row2)

n1 = length(col1)
n2 = length(col2)

S_mat = sparse(
    [i for i in 1:m1 if senses[i] != "E"],
    collect(1:n_ctr),
    [senses[i] == "L" ? 1.0 : -1.0 for i in 1:m1 if senses[i] != "E"],
    m1, n_ctr
);

n_ctr = 8


In [474]:
# Add slack coefficients to matrix
A = hcat(A, S_mat);
T = hcat(T, spzeros(m2, n_ctr))

# Add zero coeffs to objective vector
append!(c, zeros(n_ctr))

# Shift all second-stage variables to account for first-stage slack
for (k, v) in col2idx
    if v > n1
        col2idx[k] += n_ctr
    end
end
col1 = col1[1]:(m1+n_ctr)

1:22

In [475]:
senses[row1] .= "E";

### Convert W to standard form

In [476]:
# Number of non-equality constraints
n_ctr = sum(senses[row2] .!= "E")  # number of slack variables to be added

m2 = length(row2)
m1 = length(row1)

S_mat = sparse(
    [i for i in 1:m2 if senses[m1+i] != "E"],
    collect(1:n_ctr),
    [senses[m1+i] == "L" ? 1.0 : -1.0 for i in 1:m2 if senses[m1+i] != "E"],
    m2, n_ctr
);

In [477]:
# Add slack variables to W matrix
W = hcat(W, S_mat)

# Add slack variables to objective
append!(q, zeros(n_ctr));

In [478]:
senses[row2] .= "E";

In [479]:
# row2 = (m1+1):(m1+m2+n_ctr)

## Parse `.sto` file

In [480]:
name, indeps, blocks = parse_sto_file(f_sto);

In [481]:
# Extract each scenario from .sto data
S_ = Base.Iterators.product(values(indeps)..., values(blocks)...)
S = [prod(collect(s)) for s in S_]
length(S)

4096

# Construct deterministic equivalent problem

In [482]:
@time pb_data = [generate_scenario_data(A, T, W, q, h, row2idx, col2idx, s.p, s.values) for s in S][:];

  0.093582 seconds (378.60 k allocations: 67.313 MiB, 19.33% gc time)


In [483]:
T_ = [x[1] for x in pb_data];
W_ = [x[2] for x in pb_data];
q_ = [x[3] for x in pb_data];
h_ = [x[4] for x in pb_data];

In [484]:
m1, n1 = size(A)
m2, n2 = size(W)
R = length(S)

4096

In [444]:
@time U = [
    hcat(
        T_[r],
        spzeros(m2, n2*(r-1)),
        W_[r],
        spzeros(m2, n2*(R-r))
    )
    for r in 1:R
];


@time vU = vcat(U...);

  2.850399 seconds (101.25 k allocations: 4.653 GiB, 70.91% gc time)
 14.222440 seconds (414.57 M allocations: 6.193 GiB, 3.44% gc time)


In [445]:
rhs_ = vcat(b, h_...)
obj_ = vcat(c, q_...)
senses_ = vcat(senses[1:m1], repeat(senses[(m1+1):end], R))
n_ = length(obj_)
m_ = length(rhs_)
@time A_ = vcat(
    hcat(A,spzeros(m1, n2*R)),
    vU
);

rhs_l = rhs_ .- Inf .* (senses_ .== "L");
rhs_u = rhs_ .+ Inf .* (senses_ .== "G");

  0.005885 seconds (31 allocations: 13.700 MiB, 28.42% gc time)


In [446]:
size(A_), nnz(A_)

((75790, 202812), 491816)

In [447]:
airl = MPB.LinearQuadraticModel(solver)
MPB.loadproblem!(airl, A_, zeros(n_), Inf*ones(n_), obj_, rhs_l, rhs_u, :Min);

MOSEK warning 710: #1 (nearly) zero elements are specified in sparse col '' (1) of matrix 'A'.
MOSEK warning 710: #1 (nearly) zero elements are specified in sparse col '' (2) of matrix 'A'.
MOSEK warning 710: #1 (nearly) zero elements are specified in sparse col '' (4) of matrix 'A'.
MOSEK warning 710: #1 (nearly) zero elements are specified in sparse col '' (5) of matrix 'A'.
MOSEK warning 710: #1 (nearly) zero elements are specified in sparse col '' (7) of matrix 'A'.
MOSEK warning 710: #1 (nearly) zero elements are specified in sparse col '' (8) of matrix 'A'.
MOSEK warning 710: #1 (nearly) zero elements are specified in sparse col '' (10) of matrix 'A'.
MOSEK warning 710: #1 (nearly) zero elements are specified in sparse col '' (12) of matrix 'A'.
MOSEK warning 710: #1 (nearly) zero elements are specified in sparse col '' (14) of matrix 'A'.
MOSEK warning 710: #1 (nearly) zero elements are specified in sparse col '' (15) of matrix 'A'.


In [448]:
@time MPB.optimize!(airl)

  3.696303 seconds (5 allocations: 256 bytes)


In [449]:
MPB.getobjval(airl)

434.1124999999987

Dual of (DEP) writes:
\begin{align*}
    (D) \ \ \ \max_{\eta, \theta} \ \ \ & b^{T}\eta + \sum_{i} h_{i}^{T} \theta_{i}\\
    s.t. \ \ \ 
    & A^{T} \eta + \sum_{i} T_{i}^{T} \theta_{i} \leq c,\\
    & W_{i}^{T} \theta_{i} \leq p_{i} q_{i}, \ \ \ \forall i.
\end{align*}

In [450]:
@time U = [
    hcat(
        spzeros(n2, 2*m1+n1),
        spzeros(n2, m2*(r-1)),
        W_[r]',
        spzeros(n2, m2*(R-r))
    )
    for r in 1:R
];


@time vU = vcat(U...);

  2.147845 seconds (134.69 k allocations: 3.499 GiB, 47.71% gc time)
  5.912881 seconds (155.20 M allocations: 2.326 GiB, 3.05% gc time)


In [451]:
obj_ = vcat(b, -b, zeros(n1), h_...)
rhs_ = vcat(c, q_...)
senses_ = fill("L", length(rhs_))
n_ = length(obj_)
m_ = length(rhs_)
@time A_ = vcat(
    hcat(A', -A', sparse(1:n1, 1:n1, ones(n1)),transpose.(T_)...),
    vU
);

rhs_l = fill(-Inf, length(rhs_));
rhs_u = copy(rhs_);

var_l = vcat(zeros(2*m1+n1), fill(-Inf, R*m2))
var_u = fill(Inf, n_);

  0.412309 seconds (95.19 k allocations: 1.012 GiB, 18.02% gc time)


In [452]:
airl = MPB.LinearQuadraticModel(solver)
MPB.loadproblem!(airl, A_, var_l, var_u, obj_, rhs_l, rhs_u, :Max);

In [453]:
@time MPB.optimize!(airl)

  4.896854 seconds (5 allocations: 256 bytes)


In [454]:
MPB.getobjval(airl)

434.112500067542

In [455]:
MPB.getobjval(airl)

434.112500067542

# Column-generation formulation

## Create oracles

In [456]:
import Gurobi: GurobiSolver

In [506]:
GRBENV = Gurobi.Env()
spSolver = GurobiSolver(GRBENV, OutputFlag=0, Threads=1, Presolve=0, FeasibilityTol=1e-8, OptimalityTol=1e-8, Method=2)

Academic license - for non-commercial use only


GurobiSolver(Gurobi.Env(Ptr{Nothing} @0x0000000081862c10), Base.Iterators.Pairs{Symbol,Real,NTuple{6,Symbol},NamedTuple{(:OutputFlag, :Threads, :Presolve, :FeasibilityTol, :OptimalityTol, :Method),Tuple{Int64,Int64,Int64,Float64,Float64,Int64}}}(:OutputFlag=>0,:Threads=>1,:Presolve=>0,:FeasibilityTol=>1.0e-8,:OptimalityTol=>1.0e-8,:Method=>2))

In [510]:
# Instanciate sub-problems
pool = Linda.Oracle.LindaOraclePool([
    Linda.Oracle.LindaOracleMIP(
        r,
        -h_[r],
        sparse(T_[r]'),
        sparse(W_[r]'),
        fill(-Inf, n2), q_[r],  # right-hand side
        fill(:Cont, m2),
        fill(-Inf, m2), fill(Inf, m2),
        spSolver
    )
    for r in 1:R
]);

In [512]:
# Create initial columns by computing minimum cost schedule for each house
initial_cols = Linda.Column[]

@time for o in pool.oracles
    Linda.Oracle.query!(o, zeros(size(T, 2)), Inf)
    col = Linda.Oracle.get_new_columns(o)[1]
#     col = Linda.Column(1e4, zeros(size(T, 2)), true, o.index)
    push!(initial_cols, col)
end
;

  8.372339 seconds (674.32 k allocations: 165.165 MiB, 0.31% gc time)


### Create Master

In [487]:
function generate_rmp(A, R, rowlb, rowub, obj, columns, rmp_solver)
    
    rmp = MPB.LinearQuadraticModel(rmp_solver)
    
    A0 = vcat(
        spzeros(R, 2*size(A, 2)+size(A, 1)),
        hcat(A, -A, sparse(1:size(A, 1), 1:size(A, 1), ones(size(A, 1))))
    )
    A_ = hcat(
        A0,
        vcat(
            hcat([sparsevec([c.idx_subproblem], [1.0], R) for c in columns]...),
            hcat([c.col for c in columns]...)
        )
    )
    
    obj_ = vcat(obj, -obj, zeros(size(A, 1)), vcat([col.cost for col in columns]...))
    
    rowlb_ = vcat(ones(R), rowlb)
    rowub_ = vcat(ones(R), rowub)
    
#     display([rowlb_ rowub_])
    
    collb_ = zeros(2*length(obj) + size(A, 1) + length(columns))
    colub_ = Inf .* ones(2*length(obj) + size(A, 1) + length(columns))
    
#     println(size(A_))
#     println(length(collb_), "\t", length(colub_))
    
    MPB.loadproblem!(rmp, A_, collb_, colub_, obj_, rowlb_, rowub_, :Min)
    
    return rmp
end

generate_rmp (generic function with 1 method)

In [513]:
rmp = generate_rmp(A', R, c, c, -b, initial_cols, 
#     MosekSolver(
#         MSK_IPAR_PRESOLVE_USE=0,
#         MSK_IPAR_LOG=0,
#         MSK_IPAR_OPTIMIZER=3
#     )
    GurobiSolver(OutputFlag=0, Threads=1, Presolve=0, FeasibilityTol=1e-8, OptimalityTol=1e-8, Method=2)
)

mp = Linda.LindaMaster(R, length(c), c, 2*size(A, 1)+size(A, 2), initial_cols, rmp);

Academic license - for non-commercial use only


In [514]:
env = Linda.LindaEnv();
env[:verbose] = 1
env[:num_cgiter_max] = 75
env[:num_columns_max] = div(R, 10)+1

410

### Solve

In [515]:
Linda.solve_colgen!(env, mp, pool)

Itn     Primal Obj        Dual Obj         NCols    MP(s)    SP(s)   Tot(s)  BarIter  SpxIter
   0    +1.0378460e+06            -Inf      4096     0.16     1.29     1.46       10       31
   1    +9.6356429e+05            -Inf      4506     0.33     2.41     2.75       23       54
   2    +8.6678099e+05            -Inf      4916     0.54     3.72     4.26       42       81
   3    +7.6968734e+05            -Inf      5326     0.77     5.18     5.96       64      107
   4    +6.7332532e+05            -Inf      5736     1.00     6.77     7.78       84      132
   5    +5.7641607e+05            -Inf      6146     1.23     8.72     9.97      103      161
   6    +4.7935134e+05            -Inf      6556     1.47    11.06    12.55      125      183
   7    +3.8222506e+05            -Inf      6966     1.71    14.29    16.02      145      205
   8    +2.8542010e+05            -Inf      7376     1.93    18.64    20.58      162      232
   9    +1.8791773e+05            -Inf      7786     2.16   

Optimal::Status = 4

### Check that all reduced costs are indeed zero

In [409]:
MPB.optimize!(rmp)

In [464]:
A_rmp = MPB.getconstrmatrix(rmp)
redcosts = MPB.getreducedcosts(rmp)
y = MPB.getconstrduals(rmp)
x = MPB.getsolution(rmp)
c_rmp = MPB.getobj(rmp);
b_rmp = MPB.getconstrLB(rmp)

norm((c_rmp .- A_rmp'*y) .- redcosts)

5.761321961804095e-13

In [465]:
sum([nnz(sparse(col.col)) for col in mp.active_columns]) / (mp.num_columns_rmp * mp.num_constr_link)

0.6062123319425128

In [411]:
extrema(c_rmp .- A_rmp'*y)

(-7.3422654622845585e-9, 410.66499998805915)

In [412]:
extrema(redcosts)

(-7.342254804143522e-9, 410.66499998805915)

In [329]:
redcosts

94-element Array{Float64,1}:
     0.0                   
     0.0                   
     0.0                   
     0.0                   
     0.0                   
     1.451667259713812e-15 
     0.0                   
     0.0                   
    -4.440892104704481e-16 
     4.713424704661034e-17 
     0.0                   
     0.0                   
    -1.5262457964126952e-11
     ⋮                     
    20.081363111283036     
    21.12500000000079      
   432.26408355795144      
   240.0                   
    47.73591644204853      
     0.0                   
     0.0                   
     0.0                   
 12315.999999991305        
 29425.87635731887         
     0.0                   
     0.0                   

In [330]:
y = MPB.getconstrduals(rmp)
y1 = y[1:R];
y2 = y[(1+R):end];
# y2 .*= 0.0;

In [331]:
red_costs = zeros(R)
@time for r in 1:R
    # Solve sub-problem
    Linda.Oracle.query!(pool.oracles[r], y2, y1[r], farkas=false)
    # Get column
    col = Linda.Oracle.get_new_columns(pool.oracles[r])[1]
    red_costs[r] = Linda.get_reduced_cost(col, y2, y1[r], false)
end

  0.001616 seconds (107 allocations: 33.625 KiB)


In [332]:
red_costs

1-element Array{Float64,1}:
 -55337.39999999197

In [333]:
col_new = Linda.Oracle.get_new_columns(pool.oracles[1])[1]

Linda.Column{Float64,Array{Float64,1}}(-77109.4, [20784.0, 20784.0, 20784.0, 20784.0, 10392.0, 20784.0, 20784.0, 20784.0, 20784.0, 20784.0  …  0.0, 7794.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], true, false, 1, -1)

In [85]:
Linda.Oracle.query!(pool, y2, y1, log=Dict(:nsp_priced => 0))

Optimal::Status = 4

In [501]:
oracle = pool.oracles[1];

sp = MPB.LinearQuadraticModel(spSolver)

obj = oracle.costs - oracle.A_link' * y2

MPB.loadproblem!(
    sp,
    oracle.A_sub,
    oracle.col_lb,
    oracle.col_ub,
    obj,
    oracle.row_lb,
    oracle.row_ub,
    :Min
)
MPB.setvartype!(sp, oracle.vartypes)
MPB.optimize!(sp)

MethodError: MethodError: no method matching add_constrs!(::Gurobi.Model, ::Adjoint{Float64,SparseMatrixCSC{Float64,Int64}}, ::Array{Int8,1}, ::Array{Float64,1})
Closest candidates are:
  add_constrs!(::Gurobi.Model, !Matched::Array{T,1} where T, ::Array{T,1} where T, ::Array{T,1} where T, !Matched::Union{Char, Int8, Array{Char,1}, Array{Int8,1}}, !Matched::Array{T,1} where T) at /home/mathieu/.julia/packages/Gurobi/gVz8n/src/grb_constrs.jl:70
  add_constrs!(::Gurobi.Model, !Matched::Union{Array{Float64,2}, SparseMatrixCSC{Float64,Ti} where Ti<:Integer}, ::Union{Char, Int8, Array{Char,1}, Array{Int8,1}}, ::Array{Float64,1}) at /home/mathieu/.julia/packages/Gurobi/gVz8n/src/grb_constrs.jl:86
  add_constrs!(::Gurobi.Model, !Matched::Array{Int32,1}, !Matched::Array{Int32,1}, ::Array{Float64,1}, !Matched::Array{Int8,1}, !Matched::Array{Float64,1}) at /home/mathieu/.julia/packages/Gurobi/gVz8n/src/grb_constrs.jl:43

### Check that columns were properly added

In [96]:
[c.is_vertex for c in mp.active_columns]

60-element Array{Bool,1}:
  true
  true
  true
  true
  true
  true
  true
  true
  true
  true
  true
  true
  true
     ⋮
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false

In [118]:
col_ref = mp.active_columns[end]

Linda.Column{Float64,Array{Float64,1}}(0.0, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], false, true, 1, 60)

In [119]:
sparse(col_ref.col)

60-element SparseVector{Float64,Int64} with 2 stored entries:
  [26]  =  8.0
  [52]  =  6.0

In [121]:
x = MPB.getsolution(rmp)
x[mp.num_var_link+col_ref.idx_column]

0.0

In [120]:
A_rmp[:, mp.num_var_link+col_ref.idx_column]

61-element SparseVector{Float64,Int32} with 2 stored entries:
  [27]  =  8.0
  [53]  =  6.0

In [93]:
MPB.numvar(rmp)
A_rmp = MPB.getconstrmatrix(rmp)
A_rmp[:, end]

61-element SparseVector{Float64,Int32} with 2 stored entries:
  [27]  =  8.0
  [53]  =  6.0